In [ ]:
#input a vector corresponding to a probability distribution and output an index according to that distribution
function markovrand(nextv)
    if round(sum(nextv), digits = 6) != 1.0
        error(nextv, " is not a stochastic vector")
    end
    maxstates = length(nextv) - 1
    r = rand(Float64)
    current = maxstates
    for i = 0:maxstates
        if r <= nextv[i + 1]
            current = i
            break
        else
            r = r - nextv[i + 1]
        end
    end
    return(current)
end


In [ ]:
struct AugMatrix
    mat::Matrix
    init::Vector
end

struct AugDblMatrix
    mat1::Matrix
    mat2::Matrix
    init::Vector
end

In [ ]:
#output bass notes in 2*$beats notes
function ghostbassgen(beats::Int, trans::AugMatrix)
    transmat = trans.mat
    iv = trans.init
    nextv = iv
    L = []
    for j = 1:beats
        current = markovrand(nextv)
#print(current, " ")
        bincurrent = digits(current, base = 3, pad = 2)
#print(bincurrent, "\n")
        append!(L, [bincurrent[2], bincurrent[1]])
        nextv = transmat[:, current + 1]
    end
    return(L)
end

In [ ]:
#0 = rest, 1 = bass, 2 = LH
                #00    #01  #02   #10  #11  #12  #zeros
rockBassMatrix = [0.1e0 1//6  0   1//6  0.5    0 0 0 0 #00
                    0   1//3  0     0    0    0  0 0 0  #01
                    0   1//6  0.1   1//6  0.5   0 0 0 0  #02
                    0.45 1//3  0.45  1//3 0    0 0 0 0  #10
                    0.45 0    0.45  1//3  0    0 0 0 0  #11
                    0.0  0    0      0   0    0 0 0 0    #12
                    zeros(3,9)]                 #zeros

In [ ]:
sum(rockBassMatrix, dims=1)

In [ ]:
rockBassInitial = [0.1,0, 0.1, 0.4, 0.4, 0, 0, 0, 0]

In [ ]:
rockBass = AugMatrix(rockBassMatrix, rockBassInitial)

In [ ]:
#test
ghostbassgen(4, rockBass)

In [ ]:
latinNoGhostBMatrix = [0.1e0 0.2e0 0.2e0 0.5e0; #rest rest, rest bass, bass rest, bass bass
    0.4e0 0.2e0 0.6e0 0.5e0; 
    0.4e0 0.6e0 0 0; 
    0.1e0 0 0.2e0 0]

In [ ]:
latinNoGhostBInitial = [0, 0, 0, 1, 0, 0, 0, 0, 0] 

In [ ]:
latinNoGhostB = AugMatrix([latinNoGhostBMatrix[1:2, 1:2] zeros(2,1) latinNoGhostBMatrix[1:2, 3:4] zeros(2, 4)
    zeros(1, 9)
    latinNoGhostBMatrix[3:4, 1:2] zeros(2,1) latinNoGhostBMatrix[3:4, 3:4] zeros(2, 4)
    zeros(4, 9)
    ],
    latinNoGhostBInitial)

In [190]:
function lookuplatinride()::Matrix
    r = Matrix(undef, 4, 4)
    r[1,1] = ["!N![^e","[","!N![^e","[","!N![c^e","[","!N![^e","[","!N![^e","[","!N![^e","[","!N![c^e","[","!N![^e","["] #hihat eighths, trash
    #NB Windows midi doesn't have a good way of making a trash hat?  !N! defined as tenuto mark in header.abc
    r[2,1] = ["!>![^e", "[^e", "[^e", "!>![^e", "!>![c^e", "[^e", "[^e", "!>![^e", "!>![^e", "[^e", "[^e", "!>![^e", "!>![c^e", "[^e", "[^e", "!>![^e"]; #hihat 16
    r[3,1] = ["[^e", "[^e", "[_e", "[", "[c^e", "[^e", "[_e", "[", "[^e", "[^e", "[_e", "[", "[c^e", "[^e", "[_e", "[" ];  #open ands latin
    r[4,1] = ["[_e", "[", "[^e", "[^e", "[c_e", "[", "[^e", "[^e", "[_e", "[", "[^e", "[^e", "[c_e", "[", "[^e", "[^e"];  #open beats latin
    r[1,2] = ["[^g","[","[^g","[","[c^g","[","[^g","[","[^g","[","[^g","[","[c^g","[","[^g","["] #ride eighth
    r[2,2] = ["!>![^g", "[^g", "[^g", "!>![^g", "!>![c^g", "[^g", "[^g", "!>![^g", "!>![^g", "[^g", "[^g", "!>![^g", "!>![c^g", "[^g", "[^g", "!>![^g"]; #ride 16 
    r[3,2] = ["[^g^D", "[^g", "[^g", "[","[c^g^D", "[^g", "[^g", "[",  "[^g^D", "[^g", "[^g", "[", "[c^g^D",  "[^g", "[^g", "["]; #step beats latin (one e and) 
    r[4,2] = ["[^g", "[", "[^g^D", "[^g", "[c^g", "[", "[^g^D", "[^g","[^g", "[", "[^g^D", "[^g", "[c^g", "[", "[^g^D", "[^g" ]; #step ands latin (and a one)
    r[1,3] = ["[g","[","[","[","[cg","[","[","[","[g","[","[","[","[cg","[","[","["] #bell quarters
    r[2,3] = ["[","[","[g","[","[c","[","[g","[","[","[","[g","[","[c","[","[g","["] #bell ands
    r[3,3] = ["[g^D",  "[g", "[g", "[","[cg^D", "[g", "[g", "[",  "[g^D", "[g", "[g", "[", "[cg^D",  "[g", "[g", "["];  #bell and step beats latin
    r[4,3] = ["[g", "[", "[g^D", "[g", "[cg", "[", "[g^D", "[g","[g", "[", "[g^D", "[g", "[cg", "[", "[g^D", "[g" ]; #step ands bell latin 
    r[1,4] = ["[^D","[","[","[","[c^D","[","[","[","[^D","[","[","[","[c^D","[","[","["]  #step Q
    r[2,4] = ["[","[","[^D","[","[c","[","[^D","[","[","[","[^D","[","[c","[","[^D","["] #step ands
    r[3,4] = ["[^g^D", "[^g", "[^g^D", "[","[c^g^D", "[^g", "[^g^D", "[",  "[^g^D", "[^g", "[^g^D", "[", "[c^g^D",  "[^g", "[^g^D", "["]; #step 8ths latin (one e and) 
    r[4,4] = ["[^g^D", "[", "[^g^D", "[^g", "[c^g^D", "[", "[^g^D", "[^g", "[^g^D", "[", "[^g^D", "[^g", "[c^g^D", "[", "[^g^D", "[^g" ]; #step ands latin (and a one)
    return(r)
end


lookuplatinride (generic function with 1 method)

In [70]:
#input a sixteenth note bass rhythm and an 16th note ride pattern, output abc notation
function abctime16latin16ghost(L, pattern, instr)
    n = length(L)
    r = lookuplatinride()
    c = ["]/2", "F]/2", "!Q!c]/2"] #nothing, kick, ghost snare
    m = [c[L[i] + 1] for i = 1:n]
    s = ""
    for i = 1:n
        s = s * r[pattern, instr][(i - 1) % 16 + 1] * m[i]
        if i % 4 == 0
            s = s * " "
        end
        if i % 16 == 0
            s = s * "|\n"
        end
    end
    return(s)
end


abctime16latin16ghost (generic function with 1 method)

In [71]:
#test
abctime16latin16ghost(ghostbassgen(8, latinNoGhostB), 1, 1)

"!N![^eF]/2[]/2!N![^eF]/2[F]/2 !N![c^e]/2[F]/2!N![^eF]/2[]/2 !N![^e]/2[F]/2!N![^e]/2[]/2 !N![c^e]/2[F]/2!N![^e]/2[F]/2 |\n"

In [ ]:
latinUnisonMatrix = [0 0.2e0 0.2e0 0.2e0 0.2e0 0.2e0;
    0.2e0 0 0.2e0 0.2e0 0.2e0 0.2e0;
    0.2e0 0.2e0 0 0.2e0 0.2e0 0.2e0;
    0.2e0 0.2e0 0.2e0 0 0.2e0 0.2e0;
    0.2e0 0.2e0 0.2e0 0.2e0 0 0.2e0;
    0.2e0 0.2e0 0.2e0 0.2e0 0.2e0 0]

In [ ]:
latinUnisonInitial = [1//6,1//6,1//6,1//6,1//6,1//6]

In [ ]:
latinUnisonFill= AugMatrix(latinUnisonMatrix, latinUnisonInitial)

In [91]:
#input a sixteenth note rhythm in pairs, two pairs of drums, and a crescendo/descr indicator, output abc notation
function unisonfill(
  L,
  drums,
  drums2,
  crdc)
    n = length(L)
    drumlist = ["[ce]","[cA]","[cG]","[eA]","[eG]","[AG]"]
    c = ["z","z/2" * drumlist[drums] * "/2", drumlist[drums], drumlist[drums] * "/2" * drumlist[drums] * "/2"]
#print(c)
    c2 = ["z","z/2" * drumlist[drums2] * "/2", drumlist[drums2], drumlist[drums2] * "/2" * drumlist[drums2] * "/2"]
    m = vcat([c[L[i] + 1] for i = 1:div(n,2)], [c2[L[i] + 1] for i = (div(n,2) + 1):n])
# print(m)
    s = ""
    for i = 1:n
        s = s * m[i]
#print(s)
        if i % 2 == 0
            s = s * " "
        end
        if i % 8 == 0
            s = s * ""
        end
        if i % 8 == 6
            if crdc < 0
                s = s * "!>(!"
            elseif 0 < crdc
                s = s * "!<(!"
            end
        end
        if i % 8 == 7
            if crdc < 0
                s = s * "!>)!"
            elseif 0 < crdc
                s = s * "!<)!"
            end
        end
    end
    return(s)
end


unisonfill (generic function with 1 method)

In [92]:
#output rhythm in $beats pairs
function rhythmgen(beats::Int, trans::AugMatrix)
    transmat = trans.mat
    iv = trans.init
    nextv = iv
    L = []
    for j = 1:beats
        current = markovrand(nextv)
        append!(L, current)
        nextv = transmat[:, current + 1]
    end
    return(L)
end

rhythmgen (generic function with 1 method)

In [154]:
latinFillMatrix = 
    [0    0.2e0 0.1e0 0.3e0; #rest rest, rest bass, bass rest, bass bass
    0.1e0 0.2e0 0.4e0 0.5e0; 
    0.5e0 0.4e0 0.2   0.2; 
    0.4e0 0.2   0.3e0 0]

4×4 Matrix{Float64}:
 0.0  0.2  0.1  0.3
 0.1  0.2  0.4  0.5
 0.5  0.4  0.2  0.2
 0.4  0.2  0.3  0.0

In [155]:
sum(latinFillMatrix, dims=1)

1×4 Matrix{Float64}:
 1.0  1.0  1.0  1.0

In [156]:
latinFillInitial = [0.1,0.4,0.25,0.25]

4-element Vector{Float64}:
 0.1
 0.4
 0.25
 0.25

In [157]:
latinFillRhythm = AugMatrix(latinFillMatrix, latinFillInitial)

AugMatrix([0.0 0.2 0.1 0.3; 0.1 0.2 0.4 0.5; 0.5 0.4 0.2 0.2; 0.4 0.2 0.3 0.0], [0.1, 0.4, 0.25, 0.25])

In [132]:
#test
rhythmgen(8, latinFillRhythm)

8-element Vector{Any}:
 3
 1
 1
 3
 0
 2
 0
 1

In [133]:
#test
testrhythm = rhythmgen(8, latinFillRhythm)
print(testrhythm)
unisonfill(testrhythm, 3, 6, 1)

Any[2, 3, 0, 1, 2, 1, 3, 1]

"[cG][cG]/2[cG]/2 zz/2[cG]/2 [AG]z/2[AG]/2 !<(![AG]/2[AG]/2!<)!z/2[AG]/2 "

In [ ]:
dynMatrix = [
    0 0 0.2e0 0.2e0 0.1e0 0.1e0 0 0 0 0 0 0; 
    0 0 0.2e0 0.1e0 0.1e0 0.1e0 0 0 0 0 0 0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.2e0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.2e0 0.1e0; 
    0 0 0.1e0 0.2e0 0 0 0.2e0 0.1e0 0 0 0.1e0 0.2e0; 
    0 0 0.2e0 0.2e0 0 0 0.2e0 0.1e0 0 0 0.2e0 0.1e0; 
    0.1e0 0.2e0 0 0 0.1e0 0.2e0 0 0 0.2e0 0.2e0 0 0; 
    0.2e0 0.1e0 0 0 0.1e0 0.2e0 0 0 0.2e0 0.1e0 0 0; 
    0.1e0 0.2e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0;
    0.2e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0; 
    0 0 0 0 0 0 0.1e0 0.1e0 0.1e0 0.2e0 0 0; 
    0 0 0 0 0 0 0.1e0 0.1e0 0.2e0 0.2e0 0 0]


In [ ]:
dynFinal = [
    0.1e0 0.2e0 0.2e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0 0; 
    0.2e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0 0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0.2e0 0.1e0 0.2e0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0.1e0; 
    0 0 0 0 0 0 0 0 0 0 0 0; 
    0 0 0 0 0 0 0 0 0 0 0 0; 
    0 0 0 0 0 0 0 0 0 0 0 0; 
    0 0 0 0 0 0 0 0 0 0 0 0; 
    0.1e0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0; 
    0.2e0 0.1e0 0.2e0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0; 
    0 0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.2e0; 
    0 0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0.2e0 0.2e0 0.1e0]


In [ ]:
dynInitial = [0.5e-1,0.5e-1,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.5e-1,0.5e-1]
#pp>, pp<, p>, p<, mp>, mp<, mf>, mf<, f>, f<, ff>, ff<

In [ ]:
dyn = AugDblMatrix(dynMatrix, dynFinal, dynInitial)

In [ ]:
rockPatMatrix = [
    #8  Q   b   +
    0   0.4 0.4 0.4 #8 
    0.4 0   0.2 0.2   #Q
    0.2 0.2 0   0.4 #b
    0.4 0.4 0.4 0 #+
    ]

In [ ]:
sum(rockPatMatrix, dims=1)

In [ ]:
rockPatInitial = [0.4; 0.4; 0.2; 0]

In [ ]:
rockPat = AugMatrix(rockPatMatrix, rockPatInitial)

In [ ]:
rockInstMatrix = [
   #HH  RC  RB  ST 
    0.1 0.4 0.4   0.4   #HH
    0.6 0.1 0.4   0.4   #RC
    0.2 0.4 0.1   0.2   #RB
    0.1 0.1 0.1   0     #ST
]

In [ ]:
sum(rockInstMatrix, dims=1)

In [ ]:
rockInstInitial = [0.5; 0.3; 0; 0.2]

In [ ]:
rockInst = AugMatrix(rockInstMatrix, rockInstInitial)

In [ ]:
#output time, fill, time, fill with cresc/decr in abc notation
function abctune16latin16ghostUnisonfill(
  vol,
  bass,
  pat,
  inst,
  fillrhythm,
  filldrums,
  filldrums2,
  crdc)
    fp = ["!pp!","!p!","!mp!","!mf!","!f!","!ff!"]
    return("|:" * fp[vol] * abctime16latin16ghost(bass, pat, inst) * "[1" 
        * unisonfill(fillrhythm, filldrums, filldrums2, 0) * ":|2" 
        * unisonfill(fillrhythm, filldrums, filldrums2, crdc)* "\n")
end


In [115]:
#put it all together, output abc notation
include("markov-drum-utilities.jl")  #loads insert, functionName, abptest, makeppheader
import Dates
function abcsolo16latin16ghostUnisonfill(
  len::Int,
  basstrans::AugMatrix,
  filldrumstrans::AugMatrix,
  fillrhythmtrans::AugMatrix,
  dyntrans::AugDblMatrix,
  pattrans::AugMatrix,
  insttrans::AugMatrix,
  feel::String,
    )
    s = ""
    curdyn = markovrand(dyn.init)
    curpat = markovrand(pattrans.init)
    curinst = markovrand(insttrans.init)
    curfill = markovrand(filldrumstrans.init) 
    for i = 1:len
        vol = div(curdyn, 2) + 1
        bass = ghostbassgen(24, basstrans)
        pat = curpat + 1
        inst = curinst + 1
        filldrums = curfill+1
        curfill = markovrand(filldrumstrans.mat[:, curfill + 1])
        filldrums2 = curfill+1
        fillrhythm = rhythmgen(8, fillrhythmtrans)
        crdc = 2 * mod(curdyn, 2) - 1
        s = s * abctune16latin16ghostUnisonfill(vol, bass, pat, inst, fillrhythm, filldrums, filldrums2, crdc)
        if i + 1 < len
            curdyn = markovrand(dyntrans.mat1[:, curdyn + 1])
        else
            curdyn = markovrand(dyntrans.mat2[:, curdyn + 1])
        end
        curpat = markovrand(pattrans.mat[:, curpat + 1])
        curinst = markovrand(insttrans.mat[:, curinst + 1])
        curfill = markovrand(filldrumstrans.mat[:, curfill + 1])

    end
    s = insert(s, findlast(occursin("acbdefgABCDEFG"), s)-1 - 2, "!fermata!") #final fermata on last chord
    s = replace(s, "[]" => "z") #rest
    s = makeppheader("Computer-generated \\`Etude for Solo Drum Kit", String(@functionName), "$(Dates.now())", "4/4", feel, 80) * s * "|]\n"
    return(s)
end


abcsolo16latin16ghostUnisonfill (generic function with 3 methods)

In [116]:
#test
print(makeppheader("Computer-generated \\`Etude for Solo Drum Kit", String(@functionName), "$(Dates.now())", "4/4", "Rock 16th feel", 80))

%%abc-include percussions-JBH.abh

I:linebreak $

#ifdef MIDI
#define !>! "[I:volinc 50]"
#define !Q! "[I:vol 20]" 
                %%%ghost
#define !U! "[I:volinc -20]" 
                %%%unaccent
#else
#define !N! "\"^_\"" 
                %%%tenuto
#define !Q! !(.!!).! 
                %%%ghost
#define !U! !anti!  
                %%%unaccent
#endif

X:1
T:Computer-generated \`Etude for Solo Drum Kit
C:top-level scope
O:2021-11-24T16:16:01.478
M:4/4
L:1/8
Q:"Rock 16th feel" 1/4=80
K:none clef=perc
[V:1 clef=perc, stem=up]     % activate abc2xml.py map
%%voicemap drummap  % activate abcm2ps/abc2svg map
%%MIDI channel 10   % activate abc2midi map
%%MIDI program 0
%%flatbeams
%%propagate-accidentals not
%%pos ornament up
%%ornament up
%%MIDI fermatafixed


In [117]:
latinUnisonFill

AugMatrix([0.0 0.2 … 0.2 0.2; 0.2 0.0 … 0.2 0.2; … ; 0.2 0.2 … 0.0 0.2; 0.2 0.2 … 0.2 0.0], Rational{Int64}[1//6, 1//6, 1//6, 1//6, 1//6, 1//6])

In [183]:
latinsolo = abcsolo16latin16ghostUnisonfill(4, latinNoGhostB, latinUnisonFill, latinFillRhythm, dyn, rockPat, rockInst, "Latin feel")
print(latinsolo)

%%abc-include percussions-JBH.abh

I:linebreak $

#ifdef MIDI
#define !>! "[I:volinc 50]"
#define !Q! "[I:vol 20]" 
                %%%ghost
#define !U! "[I:volinc -20]" 
                %%%unaccent
#else
#define !N! "\"^_\"" 
                %%%tenuto
#define !Q! !(.!!).! 
                %%%ghost
#define !U! !anti!  
                %%%unaccent
#endif

X:1
T:Computer-generated \`Etude for Solo Drum Kit
C:abcsolo16latin16ghostUnisonfill
O:2021-11-25T18:12:46.491
M:4/4
L:1/8
Q:"Latin feel" 1/4=80
K:none clef=perc
[V:1 clef=perc, stem=up]     % activate abc2xml.py map
%%voicemap drummap  % activate abcm2ps/abc2svg map
%%MIDI channel 10   % activate abc2midi map
%%MIDI program 0
%%flatbeams
%%propagate-accidentals not
%%pos ornament up
%%ornament up
%%MIDI fermatafixed
|:!ff![^eF]/2[^e]/2[_e]/2[F]/2 [c^e]/2[^eF]/2[_eF]/2z/2 [^eF]/2[^eF]/2[_e]/2[F]/2 [c^e]/2[^eF]/2[_eF]/2z/2 |
[^e]/2[^e]/2[_e]/2z/2 [c^e]/2[^e]/2[_eF]/2z/2 [^e]/2[^eF]/2[_e]/2[F]/2 [c^eF]/2[^e]/2[_e]/2[F]/2 |
[^eF]/2[^e]/2[

In [184]:
#write a string to a file
open(f->print(f, latinsolo), "latinghosttest.abp", "w")

In [185]:
run(`cat latinghosttest.abp`)

%%abc-include percussions-JBH.abh

I:linebreak $

#ifdef MIDI
#define !>! "[I:volinc 50]"
#define !Q! "[I:vol 20]" 
                %%%ghost
#define !U! "[I:volinc -20]" 
                %%%unaccent
#else
#define !N! "\"^_\"" 
                %%%tenuto
#define !Q! !(.!!).! 
                %%%ghost
#define !U! !anti!  
                %%%unaccent
#endif

X:1
T:Computer-generated \`Etude for Solo Drum Kit
C:abcsolo16latin16ghostUnisonfill
O:2021-11-25T18:12:46.491
M:4/4
L:1/8
Q:"Latin feel" 1/4=80
K:none clef=perc
[V:1 clef=perc, stem=up]     % activate abc2xml.py map
%%voicemap drummap  % activate abcm2ps/abc2svg map
%%MIDI channel 10   % activate abc2midi map
%%MIDI program 0
%%flatbeams
%%propagate-accidentals not
%%pos ornament up
%%ornament up
%%MIDI fermatafixed
|:!ff![^eF]/2[^e]/2[_e]/2[F]/2 [c^e]/2[^eF]/2[_eF]/2z/2 [^eF]/2[^eF]/2[_e]/2[F]/2 [c^e]/2[^eF]/2[_eF]/2z/2 |
[^e]/2[^e]/2[_e]/2z/2 [c^e]/2[^e]/2[_eF]/2z/2 [^e]/2[^eF]/2[_e]/2[F]/2 [c^eF]/2[^e]/2[_e]/2[F]/2 |
[^eF]/2[^e]/2[

Process(`cat latinghosttest.abp`, ProcessExited(0))

In [186]:
#Converts ABP file to score and MIDI and displays/plays it
abptest("latinghosttest.abp")

abcm2ps-8.14.12 (2021-07-14)
File latinghosttest-ps.abc
File percussions-JBH.abh
Output written on latinghosttest.xhtml (1 page, 1 title, 55620 bytes)
4.59 June 27 2021 abc2midi
writing MIDI file latinghosttest.mid
Warning in line-char 40-0 : Track 1 Bar 19 has 9/2 units instead of 4 in repeat


latinghosttest-ps.abc:29:8: warning: Line overfull (639pt of 628pt)


Process(`cmd /C start latinghosttest.xhtml`, ProcessExited(0))

In [187]:
NoGhostSolo = abcsolo16patinst16ghost(4, rockNoGhostB, rockFill, dyn, rockPat, rockInst, "Rock 16th feel")
print(NoGhostSolo)

LoadError: UndefVarError: abcsolo16patinst16ghost not defined

In [188]:
#write a string to a file
open(f->print(f, NoGhostSolo), "rocknoghosttest.abp", "w")
#Converts ABP file to score and MIDI and displays/plays it
abptest("rocknoghosttest.abp")

LoadError: UndefVarError: NoGhostSolo not defined

In [189]:
#0 = rest, 1 = bass, 2 = LH
                #00    #01  #02   #10  #11   #12  #zeros
funkBassMatrix = [0.1e0 0.2  0     0.1  1//3 0   0 0 0 #00
                    0.2 0.2  0.2   0.2  1//3 0.1    0 0 0  #01
                    0.2 0.2  0.2   0.2  1//3 0.1    0 0 0  #02
                    0.1 0.2  0.2   0.1  0    0.3 0 0 0  #10
                    0.2 0    0.2   0.2  0    0.2 0 0 0  #11
                    0.2 0.2  0.2   0.2  0    0.3 0 0 0    #12
                    zeros(3,9)]                 #zeros

9×9 Matrix{Float64}:
 0.1  0.2  0.0  0.1  0.333333  0.0  0.0  0.0  0.0
 0.2  0.2  0.2  0.2  0.333333  0.1  0.0  0.0  0.0
 0.2  0.2  0.2  0.2  0.333333  0.1  0.0  0.0  0.0
 0.1  0.2  0.2  0.1  0.0       0.3  0.0  0.0  0.0
 0.2  0.0  0.2  0.2  0.0       0.2  0.0  0.0  0.0
 0.2  0.2  0.2  0.2  0.0       0.3  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0

In [ ]:
sum(funkBassMatrix, dims=1)

In [ ]:
funkBassInitial = [0.1, 0.2, 0.1, 0.2, 0.2, 0.2, 0, 0, 0]


In [ ]:
sum(funkBassInitial, dims=1)

In [ ]:
funkBass = AugMatrix(funkBassMatrix, funkBassInitial)

In [ ]:
funkNoGhostBMatrix = [0.1e0 1//3 0.25e0 0.5e0; 0.3e0 1//3 0.25e0 0.5e0; 0.3e0 1//3 0.25e0 0; 0.3e0 0 0.25e0 0]

In [ ]:
funkNoGhostBInitial = [0.1e0, 0.1e0, 0, 0.4e0, 0.4e0, 0, 0, 0, 0 ]

In [ ]:
funkNoGhostB = AugMatrix([funkNoGhostBMatrix[1:2, 1:2] zeros(2,1) funkNoGhostBMatrix[1:2, 3:4] zeros(2, 4)
    zeros(1, 9)
    funkNoGhostBMatrix[3:4, 1:2] zeros(2,1) funkNoGhostBMatrix[3:4, 3:4] zeros(2, 4)
    zeros(4, 9)
    ],
    funkNoGhostBInitial)

In [ ]:
funkFillLeftToRight = [0.2e0 0.1e0 0.1e0 0 0.3e0 0.2e0; 0.4e0 0.2e0 0 0.3e0 0.3e0 0.2e0; 0.1e0 0.4e0 0.2e0 0.1e0 0.1e0 0.2e0; 0.1e0 0.1e0 0.4e0 0.3e0 0.1e0 0.2e0; 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.1e0; 0.1e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0]

In [ ]:
funkFillRightToLeft = [0.4e0 0.2e0 0.1e0 0.1e0 0.3e0 0.2e0; 0.2e0 0.4e0 0.2e0 0.2e0 0.2e0 0.2e0; 0.1e0 0 0.4e0 0.1e0 0.1e0 0.2e0; 0 0.1e0 0.1e0 0.4e0 0.1e0 0.2e0; 0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0; 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0]


In [ ]:
funkFillInitial = [0.3e0,0.3e0,0.1e0,0.1e0,0.1e0,0.1e0]

In [ ]:
funkFill = AugDblMatrix(funkFillLeftToRight, funkFillRightToLeft, funkFillInitial)

In [ ]:
funkPatMatrix = [
    #8  Q   b   +
    0   0.2 0.2 0.2 #8 
    0.2 0   0.4 0.4   #Q
    0.2 0.2 0   0.4 #b
    0.6 0.6 0.4 0 #+
    ]

In [ ]:
sum(funkPatMatrix, dims=1)

In [ ]:
funkPatInitial = [0; 0.2; 0.4; 0.4]

In [ ]:
funkPat = AugMatrix(funkPatMatrix, funkPatInitial)

In [ ]:
funkInstMatrix = [
   #HH  RC  RB  ST 
    0.1 0.4 0.4   0.4   #HH
    0.6 0.1 0.4   0.4   #RC
    0.2 0.4 0.1   0.2   #RB
    0.1 0.1 0.1   0     #ST
]

In [ ]:
funkInstInitial = [0.4; 0.4; 0.2; 0]

In [ ]:
funkInst = AugMatrix(funkInstMatrix, funkInstInitial)

In [ ]:
funksolo = abcsolo16patinst16ghost(4, funkBass, funkFill, dyn, funkPat, funkInst, "Funk 16th feel")
print(funksolo)

In [ ]:
#write a string to a file
open(f->print(f, funksolo), "funkghosttest.abp", "w")

In [ ]:
run(`cat funkghosttest.abp`)

In [ ]:
#Converts ABP file to score and MIDI and displays/plays it
abptest("funkghosttest.abp")

In [ ]:
funkNoGhostsolo = abcsolo16patinst16ghost(4, funkNoGhostB, funkFill, dyn, funkPat, funkInst, "Funk 16th feel")
print(funkNoGhostsolo)

In [ ]:
#write a string to a file
open(f->print(f, funkNoGhostsolo), "funknoghosttest.abp", "w")

In [ ]:
#Converts ABP file to score and MIDI and displays/plays it
abptest("funknoghosttest.abp")